# Predicting money lost from outages

**Name(s)**: Junyi Xu and Sean Chan

**Website Link**: 

## Code

In [1]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
import datetime

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge

from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error # Built-in RMSE/MSE function.
from sklearn.model_selection import GridSearchCV

pd.options.plotting.backend = 'plotly'

In [2]:
# load outage data
outage = pd.read_excel('outage.xlsx', skiprows=5, usecols=['POSTAL.CODE', 'CAUSE.CATEGORY', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW', 'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'RES.CUSTOMERS', 'COM.CUSTOMERS', 'IND.CUSTOMERS', 'OUTAGE.START.DATE', 'OUTAGE.START.TIME'])
outage = outage.drop(index=0)
outage

,POSTAL.CODE,OUTAGE.START.DATE,OUTAGE.START.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,DEMAND.LOSS.MW,RES.PRICE,COM.PRICE,IND.PRICE,RES.SALES,COM.SALES,IND.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS
1,MN,2011-07-01 00:00:00,17:00:00,severe weather,3060,NaN,11.6,9.18,6.81,2332915,2114774,2113291,35.549073,32.225029,32.202431,2308736.0,276286.0,10673.0
2,MN,2014-05-11 00:00:00,18:38:00,intentional attack,1,NaN,12.12,9.71,6.49,1586986,1807756,1887927,30.032487,34.210389,35.727564,2345860.0,284978.0,9898.0
3,MN,2010-10-26 00:00:00,20:00:00,severe weather,3000,NaN,10.87,8.19,6.07,1467293,1801683,1951295,28.097672,34.501015,37.365983,2300291.0,276463.0,10150.0
4,MN,2012-06-19 00:00:00,04:30:00,severe weather,2550,NaN,11.79,9.25,6.71,1851519,1941174,1993026,31.994099,33.54333,34.439329,2317336.0,278466.0,11010.0
5,MN,2015-07-18 00:00:00,02:00:00,severe weather,1740,250,13.07,10.16,7.74,2028875,2161612,1777937,33.982576,36.20585,29.779498,2374674.0,289044.0,9812.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,ND,2011-12-06 00:00:00,08:00:00,public appeal,720,155,8.41,7.8,6.2,488853,438133,386693,37.212544,33.351628,29.435904,330738.0,60017.0,3639.0
1531,ND,NaN,NaN,fuel supply emergency,NaN,1650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,309997.0,53709.0,2331.0
1532,SD,2009-08-29 00:00:00,22:54:00,islanding,59,84,9.25,7.47,5.53,337874,370771,215406,36.564432,40.124517,23.311051,367206.0,65971.0,3052.0
1533,SD,2009-08-29 00:00:00,11:00:00,islanding,181,373,9.25,7.47,5.53,337874,370771,215406,36.564432,40.124517,23.311051,367206.0,65971.0,3052.0


### Framing the Problem

### Data Cleaning for Outages dataset

In [3]:
#perform probabilistic imputation on multiple columns together
def multi_prob_impute(df, cols):
    missingness = outage[cols[-1]].isna()
    fill_index = np.random.choice(outage[cols[-1]].dropna().index, missingness.sum())
    fill_values = outage.loc[fill_index, cols]
    for col in cols:
        df.loc[missingness, col] = fill_values[col].to_numpy()

# perform probabilistic imputation on single column
def single_prob_impute(df, col):
    fill_values = np.random.choice(df[col].dropna(), df[col].isna().sum())
    df.loc[df[col].isna(), col] = fill_values

In [4]:
def prob_imputate(df, col_category, sectors):
    df = df.copy()
    for category in col_category:
        for sector in sectors:
            col = '.'.join([sector, category])
            single_prob_impute(df, col)
    return df

In [5]:
def merge_sector(df, col_category, sectors):
    df = df.copy() 
    for category in col_category:
        cols = ['.'.join([sector, category]) for sector in sectors]
        df[category] = df[cols].apply(lambda row: list(row.values), axis=1)
        df = df.drop(columns=cols)
    return df

In [6]:
def conditional_mean_impute(df, target, dependent):
    return df.groupby(dependent)[target].apply(lambda x: x.fillna(x.mean()))

In [7]:
non_numeric_col = ['POSTAL.CODE', 'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'CAUSE.CATEGORY']
sector_related = ['PRICE', 'SALES', 'CUSTOMERS', 'MONEY.LOST']
sectors = ['RES', 'COM', 'IND']

outage_imputed = outage.copy()

# use probabilistic imputation to fill missing values in sector related columns
outage_imputed = prob_imputate(outage_imputed, sector_related[:-1], sectors)
multi_prob_impute(outage_imputed, ['RES.PERCEN', 'IND.PERCEN', 'COM.PERCEN'])

# use within-group mean imputation to fill missing values
outage_imputed['OUTAGE.DURATION'] = conditional_mean_impute(outage_imputed, 'OUTAGE.DURATION', 'CAUSE.CATEGORY')
outage_imputed['DEMAND.LOSS.MW'] = conditional_mean_impute(outage_imputed, 'DEMAND.LOSS.MW', 'CAUSE.CATEGORY')

outage_imputed = outage_imputed[~outage_imputed['OUTAGE.START.DATE'].isna()]
outage_imputed.loc[:, ~outage_imputed.columns.isin(non_numeric_col)] = outage_imputed.loc[:, ~outage_imputed.columns.isin(non_numeric_col)].astype(float)
outage_imputed['OUTAGE.START.DATE'] = pd.to_datetime(outage_imputed['OUTAGE.START.DATE']).dt.date

In [8]:
def money_lost(row, sector):
    return round(row['OUTAGE.DURATION']*row[f'{sector}.PERCEN']*row['DEMAND.LOSS.MW']*row[f'{sector}.PRICE'] / 60000, 2)

In [9]:
for sector in sectors:
    outage_imputed[f'{sector}.MONEY.LOST'] = outage_imputed.apply(lambda row: money_lost(row, sector), axis=1)

outage_imputed = outage_imputed.drop(columns=['DEMAND.LOSS.MW', 'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN'])
# print(outage.head().to_markdown())

outage_merge = merge_sector(outage_imputed, sector_related, sectors)
outage_explode = outage_merge.explode(sector_related).reset_index().drop(columns='index')
outage_explode['SECTOR'] = sectors*len(outage_merge)
outage_explode.loc[:, ['PRICE', 'SALES', 'CUSTOMERS', 'MONEY.LOST']] = outage_explode.loc[:, ['PRICE', 'SALES', 'CUSTOMERS', 'MONEY.LOST']].astype(float)
outage_explode = outage_explode[outage_explode['MONEY.LOST'] < 2e6]
display(outage_explode)

,POSTAL.CODE,OUTAGE.START.DATE,OUTAGE.START.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,PRICE,SALES,CUSTOMERS,MONEY.LOST,SECTOR
0,MN,2011-07-01,17:00:00,severe weather,3060.0,11.60,2332915.0,2308736.0,13010.97,RES
1,MN,2011-07-01,17:00:00,severe weather,3060.0,9.18,2114774.0,276286.0,9333.82,COM
2,MN,2011-07-01,17:00:00,severe weather,3060.0,6.81,2113291.0,10673.0,6919.25,IND
3,MN,2014-05-11,18:38:00,intentional attack,1.0,12.12,1586986.0,2345860.0,0.06,RES
4,MN,2014-05-11,18:38:00,intentional attack,1.0,9.71,1807756.0,284978.0,0.05,COM
...,...,...,...,...,...,...,...,...,...,...
4570,SD,2009-08-29,22:54:00,islanding,59.0,7.47,370771.0,65971.0,24.76,COM
4571,SD,2009-08-29,22:54:00,islanding,59.0,5.53,215406.0,3052.0,10.65,IND
4572,SD,2009-08-29,11:00:00,islanding,181.0,9.25,337874.0,367206.0,380.57,RES
4573,SD,2009-08-29,11:00:00,islanding,181.0,7.47,370771.0,65971.0,337.26,COM


# TODO
## Baseline model
### Categorical features
1. Cause cateogory
2. Sector
### Numerical features
1. Average electricity consumption (sales)
2. Duration
3. Price
4. Customers served


## Final model
### Categorical features
1. Cause cateogory
2. Date of time (Morning, Evening, Night)
3. Date (Workday or weekend)
4. Geographical region
5. Sector

### Numerical features
1. Average electricity consumption (sales)
2. Duration
3. Price
4. Customers served

In [10]:
def train(X_train, y_train, model):
    model.fit(X_train, y_train)
    return model.score(X_train,  y_train), model

def test(X_test, y_test, model):
    return model.score(X_test, y_test)

### Baseline Model

In [11]:
# baseline model
def baseline_model():
    preproc = ColumnTransformer(
        transformers=[
            ('categorical_features', OneHotEncoder(), ['CAUSE.CATEGORY', 'SECTOR'])
        ],
        remainder = 'passthrough' 
    )

    pl = Pipeline([
        ('preprocessor', preproc),
        ('lin-reg', LinearRegression())
    ])
    return pl

In [12]:
total_train_score = total_test_score = 0
num_iter = 100
dropped_cols = ['POSTAL.CODE', 'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'MONEY.LOST']
model = baseline_model()
for i in range(num_iter):
    X_train, X_test, y_train, y_test = train_test_split(outage_explode.drop(dropped_cols, axis=1), 
                                                        outage_explode['MONEY.LOST'], 
                                                        test_size=0.25)
    score, model = train(X_train, y_train, model)
    total_train_score += score
    score = test(X_test, y_test, model)
    total_test_score += score
print('train score: ', total_train_score / num_iter)
print('test score: ', total_test_score / num_iter)

train score:  0.17180298336664357
test score:  0.20517124605525375


##### Analysis
As our rmse_train is much greater than our rmse_test value (51336.40520497841 > 38759.98874640398), our baseline model is not able to generalize well

### Final Model

In [13]:
def time_of_day(time):
    if time >= datetime.time(5,0) and time < datetime.time(12, 0):
        return 'Morning'
    elif time >= datetime.time(12, 0) and time < datetime.time(17, 0):
        return 'Afternoon'
    elif time >= datetime.time(17, 0) and time < datetime.time(21, 0):
        return 'Evening'
    else:
        return 'Night'
    
us_region = {"West": ['CA', 'NV', 'UT', 'CO', 'WY', 'MT', 'ID', 'OR', 'WA', 'AK', 'HI'], 
             "Southwest": ['AZ', 'NM', 'TX', 'OK'], 
             "Midwest": ['ND', 'SD', 'NE', 'KS', 'MN', 'IA', 'MO', 'WI', 'IL', 'MI', 'IN', 'OH'], 
            "Southeast": ['AR', 'LA', 'MS', 'TN', 'AL', 'KY', 'GA', 'FL', 'NC', 'SC', 'VA', 'WV', 'DC', 'DE', 'MD'],
            "Northeast": ['NY', 'PA', 'NJ', 'CT', 'RI', 'MA', 'VT', 'NH', 'ME']}
def get_region(state):
    for region in us_region.keys():
        if state in us_region[region]:
            return region
    return np.NaN

In [14]:
outage_trans = outage_explode.copy()

outage_trans['IS.WEEKEND'] = outage_trans['OUTAGE.START.DATE'].apply(lambda d: d.weekday() > 4)
outage_trans = outage_trans.drop(['OUTAGE.START.DATE'], axis=1)

outage_trans['OUTAGE.START.TIME'] = outage_trans['OUTAGE.START.TIME'].apply(time_of_day)

outage_trans['U.S.REGION'] = outage_trans['POSTAL.CODE'].apply(get_region)
outage_trans = outage_trans.drop(['POSTAL.CODE'], axis=1)

In [15]:
class StdScalerByGroup(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        # X might not be a pandas DataFrame (e.g. a np.array)

        # Compute and store the means/standard-deviations for each column (e.g. 'c1' and 'c2'), 
        # for each group (e.g. 'A', 'B', 'C').  
        # (Our solution uses a dictionary)

        X = pd.DataFrame(X)
        self.grps_ = X.groupby(X.columns[0]).agg(['mean', 'std']).to_dict('index')
        return self

    def transform(self, X, y=None):

        try:
            getattr(self, "grps_")
        except AttributeError:
            raise RuntimeError("You must fit the transformer before tranforming the data!")
        
        # Hint: Define a helper function here!
        def standardize_group(group_df):
            group = group_df.iloc[0][group_df.columns[0]]
            return group_df[group_df.columns[1:]].apply(lambda ser: standardize(ser, group, ser.name))

        def standardize(ser, g, ser_name):
            if (ser_name, 'mean') not in self.grps_[g].keys():
                return np.full(len(ser), fill_value=np.nan)
            mean = self.grps_[g][(ser_name, 'mean')]
            std = self.grps_[g][(ser_name, 'std')]
            return (ser - mean) / std

        df = pd.DataFrame(X)
        return df.groupby(df.columns[0]).apply(standardize_group).dropna(axis=1)

In [26]:
# final model
def final_model_tuning():
    hyperparameters = {"max_iter": [300, 500, 600, 700, 800],
                      "alpha": [0.3, 0.4, 0.5, 0.6, 0.8, 0.9, 1],
                      "l1_ratio": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]}


    preproc = ColumnTransformer(
        transformers=[
            ('std-scaler', StdScalerByGroup(), ['SECTOR', 'OUTAGE.DURATION', 'SALES', 'CUSTOMERS']),
            ('one-hot', OneHotEncoder(drop='first'), ['OUTAGE.START.TIME', 'CAUSE.CATEGORY', 'IS.WEEKEND', 'SECTOR', 'U.S.REGION'])
        ],
        remainder='passthrough'
    )

    pl = Pipeline([
        ('preprocessor', preproc),
        ('grid-search', GridSearchCV(ElasticNet(), hyperparameters, scoring='r2', cv=3))
    ])
    return pl

In [29]:
X_train, X_test, y_train, y_test = train_test_split(outage_trans.drop(['MONEY.LOST'], axis=1), outage_trans['MONEY.LOST'], test_size=0.25)
tuning_model = final_model_tuning()
tuning_model.fit(X_train, y_train)
print('train score: ', tuning_model.score(X_train, y_train))
print('test score: ', tuning_model.score(X_test, y_test))
print('best parameter', tuning_model.named_steps['grid-search'].best_params_)

train score:  0.16238486193192458
test score:  0.2622639235536409
best parameter {'alpha': 0.3, 'l1_ratio': 0.6, 'max_iter': 300}


In [38]:
def final_model():
    preproc = ColumnTransformer(
        transformers=[
            ('std-scaler', StdScalerByGroup(), ['SECTOR', 'OUTAGE.DURATION', 'SALES', 'CUSTOMERS']),
            ('one-hot', OneHotEncoder(drop='first'), ['OUTAGE.START.TIME', 'CAUSE.CATEGORY', 'IS.WEEKEND', 'SECTOR', 'U.S.REGION'])
        ],
        remainder='passthrough'
    )
    pl = Pipeline([
        ('preprocessor', preproc),
        ('elastic-reg', ElasticNet(alpha=0.3, l1_ratio=0.6, max_iter=300))
    ])
    return pl

In [41]:
X_train, X_test, y_train, y_test = train_test_split(outage_trans.drop(['MONEY.LOST'], axis=1), outage_trans['MONEY.LOST'], test_size=0.25)
total_train_score = total_test_score = 0
num_iter = 50
model = final_model()
for i in range(num_iter):
    X_train, X_test, y_train, y_test = train_test_split(outage_trans.drop(['MONEY.LOST'], axis=1), 
                                                        outage_trans['MONEY.LOST'], 
                                                        test_size=0.25)
    score, model = train(X_train, y_train, model)
    total_train_score += score
    score = test(X_test, y_test, model)
    total_test_score += score
print('train score: ', total_train_score / num_iter)
print('test score: ', total_test_score / num_iter)

train score:  0.17013430005249217
test score:  0.22195186620978952


### Fairness Analysis

**Null Hypothesis**: The precision for the model in predicting money lost in residential sector is the same as the precision for the model in predicting money lost in other sector.

**Alternative Hypothesis**: The precision for the model in predicting money lost in residential sector is different from the precision for the model in predicting money lost in other sector.

In [50]:
get_R_square = lambda x: model.score(x.drop(['MONEY.LOST'], axis=1), x['MONEY.LOST'])

R_square_diff = []
observed = outage_trans.groupby('SECTOR').apply(get_R_square).diff().abs().iloc[-1]
num_iter = 1000

for _ in range(num_iter):
    df = outage_trans.copy()
    with_shuffled = df.assign(SECTOR=np.random.permutation(df['SECTOR']))
    R_square_diff.append(with_shuffled.groupby('SECTOR').apply(get_R_square).diff().abs().iloc[-1])
p_value = (R_square_diff > observed).mean()
p_value

1.0

In [53]:
fig = px.histogram(pd.DataFrame(R_square_diff, columns=['Difference in R-square value']), 
                   histnorm='probability', 
                   title='Distribution of the difference in R-square values'
                  )              
fig.add_vline(x=observed, line_color='red')
fig.show()